In [332]:
import pandas as pd
import requests
import json

In [333]:
with open("services-inclusion-2023-08-07.json", "r") as fp:
    services = json.load(fp)

In [334]:
with open("structures-inclusion-2023-08-07.json", "r") as fp:
    structures = json.load(fp)

In [447]:
def manage_dep(code):
    val = None
    if code[:2] == "20":
        if code[:3] == "200" or "201":
            val = "2A"
        elif code[:3] == "202" or "206":
            val = "2B"
    elif code[:2] == "97":
        val = code[:3]
    else:
        val = code[:2]
    return val

In [393]:
def add_to_stats(stats, inclusion_object, property_name, value):
    obj = {}
    obj[property_name] = value
    if inclusion_object["code_insee"]:
        obj["dep"] = manage_dep(inclusion_object["code_insee"])
        stats.append(obj)
    elif inclusion_object["code_postal"]:
        obj["dep"] = manage_dep(inclusion_object["code_postal"])
        stats.append(obj)
    return stats

In [394]:
def get_stats(inclusion_objects, property_name, df_ref):
    stats = []
    for inclusion_object in inclusion_objects:
        if inclusion_object[property_name]:
            if type(inclusion_object[property_name]) == list:
                for item in inclusion_object[property_name]:
                    if item in df_ref["value"].to_list():
                        stats = add_to_stats(stats, inclusion_object, property_name, item)
                    else:
                        stats = add_to_stats(stats, inclusion_object, property_name, "Non renseigné")
            else:
                if inclusion_object[property_name] in df_ref["value"].to_list():
                    stats = add_to_stats(stats, inclusion_object, property_name, inclusion_object[property_name])
                else:
                    stats = add_to_stats(stats, inclusion_object, property_name, "Non renseigné")
                    
        else:
            stats = add_to_stats(stats, inclusion_object, property_name, "Non renseigné")
    return stats

            

In [338]:
labels

,value,label,description
0,60000-rebonds,60 000 rebonds,None
1,action-logement,Action logement,None
2,adie,Adie,None
3,afpa,AFPA,None
4,agefiph,AGEFIPH,None
...,...,...,...
77,unaf,Union nationale de l’aide aux familles,None
78,unai,UNAI,None
79,unccas,Union nationale des centres communaux d’action...,None
80,unea,UNEA,None


In [339]:
def stats_df(url, inclusion_objects, property_name):
    r = requests.get(url)
    df = pd.DataFrame(r.json())
    stats = []
    stats = get_stats(inclusion_objects, property_name, df)
    stats = pd.DataFrame(stats)
    stats["value"] = 1
    stats = stats.groupby([property_name, "dep"], as_index=False).count()
    stats = stats.rename(columns={property_name: "id"})
    stats = pd.merge(stats, df[["value", "label"]].rename(columns={"value": "id"}), on="id", how="left")
    stats["label"] = stats["label"].apply(lambda x: "Non renseigné" if x != x else x)
    return stats

### Labels nationaux

In [351]:
url = "https://api.data.inclusion.beta.gouv.fr/api/v0/doc/labels-nationaux"
stats_labels = stats_df(url, structures, "labels_nationaux")
stats_labels.to_json("outputs/stats_structures_labels_nationaux.json", orient="records")

### Typologie structure

In [352]:
url = "https://api.data.inclusion.beta.gouv.fr/api/v0/doc/typologies-structures"
stats_types_structures = stats_df(url, structures, "typologie")
stats_types_structures.to_json("outputs/stats_structures_typologie.json", orient="records")

### Thématiques

In [353]:
url = "https://api.data.inclusion.beta.gouv.fr/api/v0/doc/thematiques"
stats_thematiques = stats_df(url, services, "thematiques")
stats_thematiques.to_json("outputs/stats_services_thematiques.json", orient="records")

### Typologie service

In [354]:
url = "https://api.data.inclusion.beta.gouv.fr/api/v0/doc/typologies-services"
stats_types_services = stats_df(url, services, "types")
stats_types_services.to_json("outputs/stats_services_types.json", orient="records")

### Frais

In [355]:
url = "https://api.data.inclusion.beta.gouv.fr/api/v0/doc/frais"
stats_frais = stats_df(url, services, "frais")
stats_frais.to_json("outputs/stats_services_frais.json", orient="records")

### Profils

In [356]:
url = "https://api.data.inclusion.beta.gouv.fr/api/v0/doc/profils"
stats_profils = stats_df(url, services, "profils")
stats_profils.to_json("outputs/stats_services_profils.json", orient="records")

### Modes accueil

In [357]:
url = "https://api.data.inclusion.beta.gouv.fr/api/v0/doc/modes-accueil"
stats_modes_accueil = stats_df(url, services, "modes_accueil")
stats_modes_accueil.to_json("outputs/stats_services_modes_accueil.json", orient="records")

### Générales

In [442]:
master_object = { "structure": structures, "service": services }

In [448]:
stats = []
for item_object in master_object.keys():
    for item in master_object[item_object]:
        obj = {}
        obj["type"] = item_object
        if item["code_insee"]:
            obj["dep"] = manage_dep(item["code_insee"])
            stats.append(obj)
        elif item["code_postal"]:
            obj["dep"] = manage_dep(item["code_postal"])
            stats.append(obj)        
        else:
            obj["dep"] = "Non renseigné"
            stats.append(obj)

stats = pd.DataFrame(stats)
stats["value"] = 1 
stats = stats.groupby(["dep", "type"], as_index=False).count()
stats.to_json("outputs/stats_general.json", orient="records")

### Lat/lon par département

In [449]:
def manage_geojson(master_object):
    list_dep = []
    geojson_deps = {}
    geojson = {}
    geojson["type"] = "FeatureCollection"
    geojson["properties"] = {}
    geojson["features"] = []
    for item_object in master_object.keys():
        for item in master_object[item_object]:
            if item["latitude"] and item["longitude"]:    
                obj = {}
                obj["properties"] = {}
                obj["geometry"] = {}
                obj["geometry"]["type"] = "Point"
                if item["code_insee"]:
                    obj["properties"]["dep"] = manage_dep(item["code_insee"])
                elif item["code_postal"]:
                    obj["properties"]["dep"] = manage_dep(item["code_postal"])
                obj["properties"]["source"] = item["source"]
                obj["properties"]["id"] = item["id"]
                obj["properties"]["type"] = item_object
                if item_object == "structure":
                    obj["properties"]["labels"] = item["labels_nationaux"]
                    obj["properties"]["typologie"] = [item["typologie"]]
                if item_object == "service":
                    obj["properties"]["thematiques"] = item["thematiques"]
                    obj["properties"]["types"] = item["types"]
                    obj["properties"]["frais"] = item["frais"]
                    obj["properties"]["modes_accueil"] = item["modes_accueil"]
                    obj["properties"]["profils"] = item["profils"]
                
                obj["geometry"]["coordinates"] = [item["longitude"], item["latitude"]]
                geojson["features"].append(obj)
                if obj["properties"]["dep"] not in list_dep:
                    list_dep.append(obj["properties"]["dep"])
                    geojson_deps[obj["properties"]["dep"]] = {}
                    geojson_deps[obj["properties"]["dep"]] = {}
                    geojson_deps[obj["properties"]["dep"]]["type"] = "FeatureCollection"
                    geojson_deps[obj["properties"]["dep"]]["properties"] = {}
                    geojson_deps[obj["properties"]["dep"]]["features"] = []
                geojson_deps[obj["properties"]["dep"]]["features"].append(obj)
    return geojson, geojson_deps

#### services and structures

In [450]:
geojson, geojson_deps = manage_geojson(master_object)

In [451]:
with open("outputs/inclusion_france.geojson", "w") as fp:
    json.dump(geojson, fp)

In [452]:
for item in geojson_deps.keys():
    with open(f"outputs/dep/inclusion_dep_{item}.geojson", "w") as fp:
        json.dump(geojson_deps[item], fp)